# Script To Train CNN on Google VM

In [ ]:
# Check if GPU is mounted
!nvidia-smi

In [1]:
# Load necessary packages
import numpy as np
import pandas as pd
%matplotlib inline 
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
import scipy.io
import sklearn.model_selection
import imageio

## Fit Linear Model for Benchmarking Results

In [3]:
# Load libraries
from os import listdir
from os.path import isfile, join, getsize
import numpy as np
from sklearn.linear_model import LinearRegression

# Define paths to image data
train_path = '/home/jupyter/data/training/'
val_path = '/home/jupyter/data/validation/'
test_path = '/home/jupyter/data/test/'

# Get files
train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
val_files = [f for f in listdir(val_path) if isfile(join(val_path, f))]
test_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]

# Load GDP data
y_dat = pd.read_csv("/home/jupyter/data/enhanced_gdp_data.csv")
#test_f = val_files + test_files
test_f = test_files

# Empty lists for different values
X_train = []
X_test = []
y_train_diff = []
y_test_diff = []
y_train_abs = []
y_test_abs = []

# Get necessary info from filenames and append to list
for f in train_files:
  region = f.rsplit('_')[0]
  year = int(f.rsplit('_')[2].rsplit('.')[0])
  mean = float(f.rsplit('_')[1])
  X_train.append(mean)
  y_train_diff.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),'nuts_diff'].values[0])
  y_train_abs.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),'nuts_value'].values[0])

# Get necessary info from filenames and append to list
for f in test_f:
  region = f.rsplit('_')[0]
  year = int(f.rsplit('_')[2].rsplit('.')[0])
  mean = float(f.rsplit('_')[1])
  X_test.append(mean)
  y_test_diff.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),'nuts_diff'].values[0])
  y_test_abs.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),'nuts_value'].values[0])

# Convert to numpy array
X_train = np.array([X_train])
X_test = np.array([X_test])
y_train_abs = np.array([y_train_abs])
y_test_abs = np.array([y_test_abs])
y_train_diff = np.array([y_train_diff])
y_test_diff = np.array([y_test_diff])

# Check
print('Train X: ',str(X_train[0][:3]))
print('Test X: ',str(X_test[0][:3]))

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/data/training/'

In [ ]:
# Swap axes for corrent formatting
X_train = np.swapaxes(X_train,0,-1)
X_test = np.swapaxes(X_test,0,-1)
y_train_abs = np.swapaxes(y_train_abs,0,-1)
y_test_abs = np.swapaxes(y_test_abs,0,-1)
y_train_diff = np.swapaxes(y_train_diff,0,-1)
y_test_diff = np.swapaxes(y_test_diff,0,-1)
print(X_train.shape)

In [ ]:
# Perfom linear regression for relative GDPs
reg = LinearRegression().fit(X_train, y_train_diff)
r_square = reg.score(X_train, y_train_diff)
print('R Square Difference: ',str(r_square))

# Get predictions
pred = reg.predict(X_test)

# Get prediction accuracy
mse = np.mean((y_test_diff-pred)**2)
mae = np.mean(np.abs(y_test_diff-pred))
print('MSE Difference: ',str(mse))
print('MAE Difference: ',str(mae))

In [ ]:
# Perfom linear regression for absolute GDPs
reg = LinearRegression().fit(X_train, y_train_abs)
r_square = reg.score(X_train, y_train_abs)
print('R Square Absolute: ',str(r_square))

# Get predictions
pred = reg.predict(X_test)

# Get prediction accuracy
mse = np.mean((y_test_abs-pred)**2)
mae = np.mean(np.abs(y_test_abs-pred))
print('MSE Absolute: ',str(mse))
print('MAE Absolute: ',str(mae))

## Sample CNN Architectures

In [ ]:
# Load libraries to train CNN models with Tensorflow
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import GaussianDropout
from keras.layers import GaussianNoise
from keras.regularizers import l1, l2, l1_l2
from keras.optimizers import SGD, Adam, Nadam, Adamax, Adadelta, RMSprop
from keras.applications import VGG19, ResNet50, ResNet50V2, InceptionResNetV2, MobileNetV2
from keras.callbacks import EarlyStopping
import scipy
import pandas as pd
import numpy as np
from datetime import date
from os import listdir
import itertools
from os.path import isfile, join, getsize
#Image.MAX_IMAGE_PIXELS = 213437725

In [ ]:
# Define input shape
in_shape = (224,224,3)
out_shape = 1

## Define CNN Models used for the thesis

In [ ]:
# Very shallow CNN
def xs_model(in_shape = (224,224,3), out_shape=1):
    xs_model = Sequential()

    xs_model.add(Conv2D(32, 3, padding='same', input_shape=in_shape, activation='relu'))
    xs_model.add(Flatten())
    #xs_model.add(Dense(128, activation='relu'))
    xs_model.add(Dense(units=out_shape,activation='linear'))
    
    return xs_model

In [ ]:
# Very shallow CNN
def s_model(in_shape = (224,224,3),out_shape=1):
    s_model = Sequential()

    s_model.add(Conv2D(32, 3, padding='same', input_shape=in_shape, activation='relu'))
    s_model.add(Conv2D(32, 3, padding='same', activation='relu'))
    s_model.add(BatchNormalization())
    s_model.add(MaxPooling2D(pool_size = (2,2)))

    s_model.add(Flatten())
    s_model.add(Dense(128, activation='relu', activity_regularizer=l1(0.001)))
    s_model.add(Dropout(0.5))
    s_model.add(Dense(units=out_shape,activation='linear'))
    
    return s_model

In [ ]:
# Med Shallow Model
def m_model(in_shape=(224,224,3),out_shape=1):
    m_model = Sequential()

    m_model.add(Conv2D(16, 3, padding='same', input_shape=in_shape, activation='relu'))
    m_model.add(Conv2D(16, 3, padding='same', activation='relu'))
    m_model.add(BatchNormalization())
    m_model.add(MaxPooling2D(pool_size = (2,2)))

    m_model.add(Conv2D(32, 3, padding='same', activation='relu'))
    m_model.add(Conv2D(32, 3, padding='same', activation='relu'))
    m_model.add(BatchNormalization())
    m_model.add(MaxPooling2D(pool_size = (2,2)))

    m_model.add(Flatten())
    m_model.add(Dense(128, activation='relu', activity_regularizer=l1(0.001)))
    m_model.add(Dropout(0.5))

    m_model.add(Dense(256, activation='relu',activity_regularizer=l1(0.001)))
    m_model.add(Dropout(0.5))
    m_model.add(Dense(units=out_shape,activation='linear'))
    
    return m_model

In [ ]:
# Transfer Learning Mobile Model
def mobile_model(in_shape=(224,224,3),out_shape=1):
    base_model = MobileNetV2(input_shape=in_shape,include_top=False)

    x=base_model.output
    x=Flatten()(x)
    x=Dense(32,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    x=Dropout(0.2)(x) #Dropout
    x=Dense(32,activation='relu')(x) #dense layer 3
    x=Dropout(0.2)(x) #Dropout
    preds=Dense(units=out_shape, activation = 'linear')(x)
    mobile_model=Model(inputs=base_model.input,outputs=preds)

    for layer in mobile_model.layers[:-26]:
        layer.trainable = False
    
    return mobile_model

In [ ]:
# Transfer Learning Inception Model
def inception_model(in_shape=(224,224,3),out_shape=1):
    base_model = InceptionResNetV2(input_shape=in_shape,include_top=False)

    x=base_model.output
    x=Flatten()(x)
    x=Dense(32,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    x=Dropout(0.2)(x) #Dropout
    x=Dense(32,activation='relu')(x) #dense layer 3
    x=Dropout(0.2)(x) #Dropout
    preds=Dense(units=out_shape, activation = 'linear')(x)
    inception_model=Model(inputs=base_model.input,outputs=preds)
    
    return inception_model

# Training Pipeline for CNN Fitting

In [ ]:
# Plot histogram of image size in different sets
train_path = "/home/jupyter/data/training/"
val_path = "/home/jupyter/data/validation/"
test_path = "/home/jupyter/data/test/"

train_files = [getsize(join(train_path, f)) for f in listdir(train_path) if isfile(join(train_path, f))]
val_files = [getsize(join(val_path, f)) for f in listdir(val_path) if isfile(join(val_path, f))]
test_files = [getsize(join(test_path, f)) for f in listdir(test_path) if isfile(join(test_path, f))]

pyplot.hist(train_files,bins=1000)
pyplot.show()

pyplot.hist(val_files,bins=1000)
pyplot.show()

pyplot.hist(test_files,bins=1000)
pyplot.show()

## Pipeline of functions to automate the training

In [ ]:
# Create DataFrame to load the data
def data_load(data_type, pred = 'nuts_value'):
  
  # Define path to image files
  train_path = "/home/jupyter/data/training/"
  val_path = "/home/jupyter/data/validation/"
  test_path = "/home/jupyter/data/test/"

  # Get files
  train_files = [f for f in listdir(train_path) if isfile(join(train_path, f)) and getsize(join(train_path, f)) > 100]
  val_files = [f for f in listdir(val_path) if isfile(join(val_path, f)) and getsize(join(val_path, f)) > 100]
  test_files = [f for f in listdir(test_path) if isfile(join(test_path, f)) and getsize(join(test_path, f)) > 100]

  # NUT USED - for time series experiments
  if pred == 'diff':
    y_dat = pd.read_csv("/home/jupyter/data/gdp_time_series.csv")
  else:
    y_dat = pd.read_csv("/home/jupyter/enhanced_gdp_data.csv")

  train_y = []
  val_y = []
  test_y = []

  # Get the paths and GDP values for training data
  for f in train_files:
    if pred == 'diff':
      year = f.rsplit('_')[1].rsplit('.')[0]
      region = f.rsplit('_')[0]
      train_y.append(y_dat.loc[(y_dat['nuts2']==region),str('diff_'+year)].values[0])
    else:
      year = int(f.rsplit('_')[2].rsplit('.')[0])
      region = f.rsplit('_')[0]
      train_y.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),pred].values[0])

  # Get the paths and GDP values for validation data
  for f in val_files:
    if pred == 'diff':
      year = f.rsplit('_')[1].rsplit('.')[0]
      region = f.rsplit('_')[0]
      val_y.append(y_dat.loc[(y_dat['nuts2']==region),str('diff_'+year)].values[0])
    else:
      year = int(f.rsplit('_')[2].rsplit('.')[0])
      region = f.rsplit('_')[0]
      val_y.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),pred].values[0])

  # Get the paths and GDP values for test data
  for f in test_files:
    if pred == 'diff':
      year = f.rsplit('_')[1].rsplit('.')[0]
      region = f.rsplit('_')[0]
      test_y.append(y_dat.loc[(y_dat['nuts2']==region),str('diff_'+year)].values[0])
    else:
      year = int(f.rsplit('_')[2].rsplit('.')[0])
      region = f.rsplit('_')[0]
      test_y.append(y_dat.loc[(y_dat['nuts2']==region)&(y_dat['year']==year),pred].values[0])

  # Scale the output GDP data
  train_y = np.array(train_y)
  train_y = train_y.reshape(len(train_y),1)
  val_y = np.array(val_y)
  val_y = val_y.reshape(len(val_y),1)
  test_y = np.array(test_y)
  test_y = test_y.reshape(len(test_y),1)
  scaler = StandardScaler()
  scaler.fit(train_y)
  train_y = scaler.transform(train_y)
  val_y = scaler.transform(val_y)
  test_y = scaler.transform(test_y)
  
  # Put info into DF so that Keras can read in the data automatically
  train_df = pd.DataFrame({'y':train_y[:,0].tolist(),'file':train_files})
  val_df = pd.DataFrame({'y':val_y[:,0].tolist(),'file':val_files})
  test_df = pd.DataFrame({'y':test_y[:,0].tolist(),'file':test_files})

  # Return DF per split and the scaler for later
  return train_df, val_df, test_df, scaler

In [ ]:
# define cnn model
def define_model(lr,in_shape, out_shape,optim,model):
	
    # Define new model
    mod = Sequential()
    if model == 'xs_model':
        mod = xs_model(in_shape=in_shape)
    elif model == 's_model':
        mod = s_model(in_shape=in_shape)
    elif model == 'm_model':
        mod = m_model(in_shape=in_shape)
    elif model == 'inception_model':
        mod = inception_model(in_shape=in_shape)
    else:
        mod = mobile_model(in_shape=in_shape)
        
    
    # compile model with different optimising functions and learning rates
    if optim == 'adam':
        opt = Adam(lr=lr)
    elif optim == 'sgd':
        opt = SGD(lr=lr)
    elif optim == 'rmsprob':
        opt = RMSprop(lr=lr, rho=0.9)
    else:
        opt = Adamax(lr=lr)
    
    # Compile the model
    mod.compile(optimizer=opt, loss='mean_squared_error', metrics=['mse','mae'])
 
    return mod

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history,no_layers,in_shape,batch_size,epochs,mse,mae,transfer,lr,optim,comment,data_type,pred):
	# Save the Diagnostics and Training plot to drive if mounted, otherwise comment out

	# plot loss
	pyplot.subplot(211)
	pyplot.title('Model MSE')
	pyplot.plot(history.history['mean_squared_error'], color='blue', label='train')
	pyplot.plot(history.history['val_mean_squared_error'], color='orange', label='val')
	pyplot.xlabel("")
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Model MAE')
	pyplot.plot(history.history['mean_absolute_error'], color='blue', label='train')
	pyplot.plot(history.history['val_mean_absolute_error'], color='orange', label='val')
	# save plot to file
	day = date.today()
	filename = str(day)+"_"+str(in_shape[0])+"_"+str(in_shape[2])+"_"+str(epochs)+"_"+str(batch_size)
	pyplot.savefig('/gdrive/My Drive/ThesisData/cnn_results/'+filename + '_plot.png')
	pyplot.close()
  
	# write diagnostics to results file
	myrow = ['\n'+str(day),str(no_layers),str(batch_size),str(epochs),str(in_shape[0]),str(in_shape[2]),
	         str(history.history['mse'][-1]),str(history.history['mae'][-1]),
					     str(history.history['val_mse'][-1]),str(history.history['val_mae'][-1]),
							   str(mse),str(mae),str(transfer),str(lr),str(optim),str(comment),str(data_type),str(pred)]
	
	myrow = ','.join(myrow)
	filepath = '/home/jupyter/cnn_results_diff.csv'
	with open(filepath,'a') as fd:
		fd.write(myrow)

In [ ]:
# run the test harness for evaluating a model
def run_test_harness(comment,transfer,optim,model,data_type,lr=0.0001,batch_size = 10,epochs = 50, target_size = 128, pred = 'nuts_value'):
  
  # load dataset
  train_df, val_df, test_df, scaler = data_load(data_type=data_type, pred = pred)
  
  # create data generator
  train_datagen = ImageDataGenerator()#rescale=(1.0/255))
  val_datagen = ImageDataGenerator()#rescale=(1.0/255))
  test_datagen = ImageDataGenerator()#rescale=(1.0/255))

  # Define early stopping
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=30, restore_best_weights=True)

  # File Directories
  train_path = "/home/jupyter/data/training/"
  val_path = "/home/jupyter/data/validation/"
  test_path = "/home/jupyter/data/test/"

  # Infer color mode and input shape - only night was used
  if 'day' in data_type or 'night' in data_type or data_type == 'subsample':
    color_mode = 'rgb'
    in_shape = (target_size,target_size,3)
  else:
    color_mode = 'rgba'
    in_shape = (target_size,target_size,4)

  print(in_shape)

  # prepare iterators and give them the prepared DF with image paths and GDP values
  train_it = train_datagen.flow_from_dataframe(train_df,
                                               directory=train_path,
                                               x_col = 'file',
                                               y_col = 'y',
                                               target_size = (target_size,target_size),
                                               batch_size = batch_size,
                                               class_mode = 'raw',
                                               color_mode = color_mode)
  
  val_it = val_datagen.flow_from_dataframe(val_df, 
                                           directory=val_path,
                                           x_col = 'file',
                                           y_col = 'y',
                                           target_size = (target_size,target_size),
                                           batch_size = batch_size,
                                           class_mode = 'raw',
                                           color_mode = color_mode)
  
  test_it = test_datagen.flow_from_dataframe(test_df, 
                                             directory=test_path,
                                             x_col = 'file',
                                             y_col = 'y',
                                             target_size = (target_size,target_size),
                                             batch_size = batch_size,
                                             class_mode = 'raw',
                                             color_mode = color_mode)
  
  # define model
  transfer = model
  print('Model: '+model)
  model = define_model(lr=lr,in_shape=in_shape,out_shape=1,optim=optim,model=model)
  
  # Get number of layers
  no_layers = len(model.layers)
  
  # fit model
  history = model.fit_generator(train_it, 
                                steps_per_epoch=len(train_it),
                                validation_data=val_it, 
                                validation_steps=len(val_it),
                                callbacks = [es], 
                                epochs=epochs, 
                                verbose=2)
  
  # evaluate model on unscaled GDP values
  test_preds = model.predict_generator(test_it, steps=len(test_it), verbose=0)
  # Inverse transformation to get unscaled predictions
  unscaled_preds = scaler.inverse_transform(test_preds)
  
  # Get true GDP values
  true_vals = test_it.labels
  # Inverse transformation for true values
  true_vals = scaler.inverse_transform(true_vals)
  
  # Compute unscaled MSE and MAE
  mse = np.mean((true_vals-unscaled_preds)**2)
  mae = np.mean(abs(true_vals-unscaled_preds))
  print('> mse=%.3f, mae=%.3f' % (mse, mae))
  
  # Save training plot and diagnostics
  summarize_diagnostics(history=history,
                        no_layers=no_layers,
                        in_shape = in_shape,
                        batch_size=batch_size,
                        epochs=len(history.history['loss']),
                        mse=mse,mae=mae,
                        transfer=transfer,lr=lr,
                        optim=optim,
                        comment=comment,
                        data_type=data_type,
                        pred = pred)
  
  return model, test_it, scaler

In [ ]:
# Prepare iteration with all models, optimiser, and learning rates
models = ["xs_model","s_model","m_model","inception_model","mobile_model"]
optims = ["sgd","adam","rmsprob","adamax"]
lrs = [0.001,0.0001,0.00001,0.000001]
opt_list = list(itertools.product(models,optims,lrs))

# Number of models to train
print(len(opt_list))

In [ ]:
# Start training iteration
i = 0
in_shape = (224,224,3)
for combi in opt_list[:]:
  print('Optim: '+combi[1]+', LR: '+str(combi[2]))
  print(i)
  model, test_it, scaler = run_test_harness("iterated nn","missing",combi[1],combi[0],data_type='viirs_night',lr=combi[2],batch_size=16,epochs=1000,target_size=in_shape[0], pred='nuts_diff')
  i = i+1

## Detailed Prediction Results for best CNNN models

In [ ]:
# Detailed Prediction Analysis - Relative
i = 0
test_preds = []
test_files = []
test_true = []

# Repeat 30 times
while i < 30:

    print('Iteration: '+str(i))
    
    # Train best CNN model for absolute GDP
    model, test_it, scaler = run_test_harness("iterated nn","missing",'adam','mobile_model',data_type='viirs_night',lr=0.0001,batch_size=8,epochs=1000,target_size=in_shape[0], pred='nuts_diff')
    
    # Rescale results
    temp_preds = model.predict_generator(test_it)
    temp_preds = scaler.inverse_transform(temp_preds)
    temp_files = test_it.filenames
    temp_true = test_it.labels
    temp_true = scaler.inverse_transform(temp_true)
    
    # Add results to lists
    test_preds.extend(list(temp_preds[:,0]))
    test_files.extend(list(temp_files))
    test_true.extend(list(temp_true))
    i = i+1
    
    # Transform all into a DF
    res_dict = {'test_preds':test_preds, 'test_files':test_files, 'test_true_vals':test_true}
    result_dat = pd.DataFrame(res_dict)

    # Get Country
    result_dat['country'] = result_dat['test_files'].str[:2]

    # Compute Error
    result_dat['se'] = (result_dat['test_true_vals']-result_dat['test_preds'])**2
    result_dat['abs_error'] = abs(result_dat['test_true_vals']-result_dat['test_preds'])
    
    # Country level statistics
    country_dat = result_dat.groupby(['country']).agg({'se':'mean','abs_error':'mean','test_files':'size'}).reset_index()
    
    # Save
    result_dat.to_csv("result_preds_relative.csv", index=False)
    country_dat.to_csv("result_country_relative.csv", index=False)

In [ ]:
# Detailed Prediction Analysis - Relative
i = 0
test_preds = []
test_files = []
test_true = []

# Repeat 30 times
while i < 30:
    print('Iteration: '+str(i))
    
    # Train best model for relative GDP
    model, test_it, scaler = run_test_harness("detailed analysis","missing","rmsprob","inception_model",data_type='viirs_night',lr=0.00001,batch_size=8,epochs=1000,target_size=in_shape[0], pred='nuts_value')
    
    # Rescale results
    temp_preds = model.predict_generator(test_it)
    temp_preds = scaler.inverse_transform(temp_preds)
    temp_files = test_it.filenames
    temp_true = test_it.labels
    temp_true = scaler.inverse_transform(temp_true)
    
    # Save to lists
    test_preds.extend(list(temp_preds[:,0]))
    test_files.extend(list(temp_files))
    test_true.extend(list(temp_true))
    i = i+1
    
    # Transform all into a DF
    res_dict = {'test_preds':test_preds, 'test_files':test_files, 'test_true_vals':test_true}
    result_dat = pd.DataFrame(res_dict)

    # Get Country
    result_dat['country'] = result_dat['test_files'].str[:2]

    # Compute Error
    result_dat['se'] = (result_dat['test_true_vals']-result_dat['test_preds'])**2
    result_dat['abs_error'] = abs(result_dat['test_true_vals']-result_dat['test_preds'])
    
    # Compute country stats
    country_dat = result_dat.groupby(['country']).agg({'se':'mean','abs_error':'mean','test_files':'size'}).reset_index()
    
    # Save
    result_dat.to_csv("result_preds_abs.csv", index=False)
    country_dat.to_csv("result_country_abs.csv", index=False)